<a href="https://colab.research.google.com/github/starcell/sejong-ml/blob/main/mnist_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# Check if CUDA is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Hyperparameters
batch_size = 64
learning_rate = 0.01
epochs = 10

In [ ]:
# MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

In [ ]:
# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Define a simple CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net().to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        
        loss = criterion(output, target)
        loss.backward()
        
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')

In [ ]:
# Testing function
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += criterion(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)

    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy * 100:.0f}%)\n')

In [ ]:
# Run training and testing loops
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

확인

In [ ]:
import random
import matplotlib.pyplot as plt

In [ ]:
# Get a random sample from the test set
index = random.randint(0, len(test_dataset) - 1)
sample_data, sample_target = test_dataset[index]
sample_data = sample_data.to(device)

In [ ]:
# Make prediction
with torch.no_grad():
    output = model(sample_data.unsqueeze(0))  # Add batch dimension
    prediction = output.argmax(dim=1).item()

In [ ]:
# Display the result
plt.imshow(sample_data.cpu().numpy().squeeze(), cmap='gray')
plt.title(f'Actual: {sample_target}, Predicted: {prediction}')
plt.show()

In [ ]:
test_batch_size = 10000
test_loader = torch.utils.data.DataLoader(
  test_dataset,  batch_size=test_batch_size, shuffle=False)

In [ ]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
with torch.no_grad():
  output = model(example_data.to(device))

In [ ]:
# fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Prediction: {}".format(
    output.data.max(1, keepdim=True)[1][i].item()))
  plt.xticks([])
  plt.yticks([])
# fig

In [ ]:
predictions = output.data.max(1, keepdim=True)[1] # [0].item()

In [ ]:
predictions.size()

In [ ]:
predictions.size()[0]

In [ ]:
predictions[0].item()

In [ ]:
test_dataset.targets[0].item()

In [ ]:
wrong_predicts = []
for i in range(predictions.size()[0]):
    if not predictions[i].item() == test_dataset.targets[i].item():
        wrong_predicts.append(i)

In [ ]:
len(wrong_predicts)

In [ ]:
# wrong_predicts

for i in range(6):
  id = wrong_predicts[i]
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[id][0], cmap='gray', interpolation='none')
  plt.title("GT: {}, Pred: {} ".format(
    test_dataset.targets[id],
    output.data.max(1, keepdim=True)[1][id].item()))
  plt.xticks([])
  plt.yticks([])